# Make clf

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functions.date_split import split_month_test

%matplotlib inline

from sklearn.model_selection import cross_val_score

np.random.seed(seed=42)

from score_submission import our_scorer, scorer
from sklearn.pipeline import Pipeline
from sklearn import datasets, linear_model
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestRegressor
import pickle

In [2]:
import functions.CreatingNewDataframe as CreatingNewDataframe
import functions.GenerateFis as Features
import functions.GenerateMeansWithOffset as MeansOffset
from functions.pipeline_steps import *

from functions.date_split import split_month_test
from score_submission import our_scorer, scorer

In [3]:
df_initial = pd.read_csv('train.csv.zip', parse_dates=['DATE'])
ids = np.unique(df_initial.ATM_ID)

In [4]:
drop_days = 70
train_days = 60

In [6]:
df1, df2, df3, df4, df5 = Features.GenerateFisSasha(df_initial)

In [7]:
_, train_y = split_month_test(df_initial, train_days)
_, train1_X = split_month_test(df5, train_days)
#_, train2_X = split_month_test(df1, train_days)
#_, train3_X = split_month_test(df1, train_days)
#_, train4_X = split_month_test(df1, train_days)
#_, train5_X = split_month_test(df1, train_days)

In [8]:
#train1_X = train1_X.drop(['CLIENT_OUT'], axis=1)
train1_X = train1_X.dropna(subset=train1_X.columns.drop('CLIENT_OUT'))

In [9]:
clf = Pipeline([
    ('lasso', WrapperForEstimator(linear_model.Lasso())), #predict step
    ("submission", FinalStepForSubmission()),
])

In [10]:
clf.fit(train1_X, train_y[['CLIENT_OUT']])

WrapperForEstimator transform called
Transform from estimator Finished


/home/progiv/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


Pipeline(memory=None,
     steps=[('lasso', WrapperForEstimator(estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False))), ('submission', FinalStepForSubmission())])

In [11]:
y_pred = clf.predict(train1_X)

WrapperForEstimator transform called
Transform from estimator Finished
300
FinalStepForSubmission Finished


In [12]:
with open("model_to_run.pickle", "wb") as file:
    pickle.dump(clf, file)